In [9]:
# import math
import numpy as np
import scipy.spatial.distance
import scipy.signal
from skimage.segmentation import slic
from skimage.util import img_as_float
from scipy.optimize import minimize

import os
import scipy.io
import cv2
import pywt
import numpy as np
import numpy as np
# from mmdet.core import bbox_iou

from skimage import io
# import itertools

In [10]:
File_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach8/Input_Image_Dataset/bounding_boxes_with_multi_boxes'
HS_files = sorted(os.listdir(File_path))
print("HS_files", HS_files)
# num = 0;
save_path = '/home/nitk/Desktop/Raj_M.Tech_Project/Approach8/Input_Image_Dataset/bounding_box_single_box'

HS_files ['0022.jpg']


In [11]:
def mkdir(dir_path):
    if not os.path.exists(dir_path):
        os.makedirs(dir_path)

In [47]:
def calculate_iou(box1, box2):
    # Calculate the coordinates of the intersection rectangle
    x1 = max(box1[0], box2[0])
    y1 = max(box1[1], box2[1])
    x2 = min(box1[2], box2[2])
    y2 = min(box1[3], box2[3])
    
    # Calculate the area of intersection rectangle
    intersection_area = max(0, np.abs(x2 - x1 + 1)) * max(0, np.abs(y2 - y1 + 1))
    print('intersection area', intersection_area)
    
    # Calculate the areas of both bounding boxes
    box1_area = (box1[2] - box1[0] + 1) * (box1[3] - box1[1] + 1)
    print('box1_area', box1_area)
    box2_area = (box2[2] - box2[0] + 1) * (box2[3] - box2[1] + 1)
    print('box2_area', box2_area)

    # Calculate the IoU
    iou = intersection_area / float(np.abs(box1_area) + np.abs(box2_area) - np.abs(intersection_area))
    
    return np.abs(iou)

In [39]:
def combine_bboxes(box1, box2):
    iou = calculate_iou(box1, box2)
    print('iou', iou)
    if iou > 0.1:
        # Boxes are overlapping, merge them
        combined_box = [min(box1[0], box2[0]), min(box1[1], box2[1]), max(box1[2], box2[2]), max(box1[3], box2[3])]
        return combined_box
    else:
        # Boxes are not overlapping, do not merge
        return None

In [ ]:
# def merge_bboxes(box1, box2):
#     iou = bbox_iou(box1[np.newaxis, :], box2[np.newaxis, :])
#     if iou < 0:
#         # Boxes are not overlapping
#         merged_box = [min(box1[0], box2[0]), min(box1[1], box2[1]), max(box1[2], box2[2]), max(box1[3], box2[3])]
#         return merged_box
#     else:
#         # Boxes are overlapping, do not merge
#         return None

In [56]:
file_list = os.listdir(File_path)
for file in file_list:
    file_name = file.rsplit('.', 1)
    input_img = os.path.join(File_path, file)
    #  Load the saliency map
    
    saliency_map = cv2.imread(input_img, cv2.IMREAD_GRAYSCALE)

    # print('saliency_map', saliency_map)

    # Threshold the saliency map to obtain a binary mask
    # print('before threshold', saliency_map)
    threshold =  128  # adjust this threshold as needed
    mask = (saliency_map > threshold).astype(np.uint8)
    

    kernel = np.ones((6,6),np.uint8)
    mask = cv2.erode(mask,kernel,iterations=1)

    # Find the connected components in the binary mask
    components, labels = cv2.connectedComponents(mask)

    # Initialize a list to store the bounding boxes and scores of each component
    boxes = []
    scores = []

    # Loop over each component and compute its bounding box and score
    for i in range(1, np.amax(components) + 1):
        # Extract the binary mask for the current component
        component_mask = (labels == i).astype(np.uint8)

        # Compute the bounding box of the component
        bbox = cv2.boundingRect(component_mask)

        # Compute the score of the component as the maximum saliency value within the component
        if np.count_nonzero(component_mask) > 0:
            score = np.max(saliency_map[component_mask == 1])
        else:
            score = 0

        # Append the bounding box and score to the list
        boxes.append(bbox)
        scores.append(score)

    # Apply non-maximum suppression to the list of bounding boxes and scores
    overlap_thresh = 0.5  # adjust this threshold as needed
    scores = np.array(scores)
    boxes = np.array(boxes)
    selected_indices = []

    # Flatten the scores array before performing argsort
    order = scores.ravel().argsort()[::-1]
    areas = (boxes[:, 2] * boxes[:, 3])

    if np.any(areas == 0):
        areas[areas == 0] = 1

    # Loop over the indices in the order determined by the scores and compute the overlap of each bounding box with the previously selected bounding boxes
    while order.size > 0:
        # Select the bounding box with the highest score and add it to the list of selected boxes
        i = order[0]
        selected_indices.append(i)

        # Compute the overlap of the selected bounding box with the other bounding boxes
        xx1 = np.maximum(boxes[i, 0], boxes[order[1:], 0])
        yy1 = np.maximum(boxes[i, 1], boxes[order[1:], 1])
        xx2 = np.minimum(boxes[i, 0] + boxes[i, 2], boxes[order[1:], 0] + boxes[order[1:], 2])
        yy2 = np.minimum(boxes[i, 1] + boxes[i, 3], boxes[order[1:], 1] + boxes[order[1:], 3])
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        overlap = (w * h) / areas[order[1:]]

        # Remove bounding boxes with overlap greater than the overlap threshold
        inds = np.where(overlap <= overlap_thresh)[0]
        order = order[inds + 1]

    # areas = (boxes[:, 2] - boxes[:, 0] + 1) * (boxes[:, 3] - boxes[:, 1] + 1)

    # # Sort the boxes based on their area in descending order
    # sorted_indices = np.argsort(areas)[::-1]
    # sorted_boxes = boxes[sorted_indices]


    # Loop over the selected bounding boxes and draw them on the image
    img_rgb = cv2.cvtColor(saliency_map, cv2.COLOR_GRAY2RGB)
    area = img_rgb[279:467, 31:768]
    gray_roi = np.ones((188, 737), dtype=np.uint8)
    white_roi = 255 - gray_roi
    img_rgb[279:467, 31:768] = cv2.cvtColor(white_roi, cv2.COLOR_GRAY2BGR)
    # for i in selected_indices:
    #     x, y, w, h = boxes[i]
    #     cv2.rectangle(img_rgb, (x, y), (x + w, y + h), (0, 255, 0), 1)

    # combined_boxes = combine_bboxes(boxes[1], boxes[2])
    # x, y, w, h = boxes[i]
    # cv2.rectangle(img_rgb, (31, 279), (768, 467), (0, 255, 0), 1)

    # print('combined boxes', combined_boxes)
    print('boxes[0]', boxes[0])
    print('boxes[1]', boxes[1])
    # print('selected indices', selected_indices)
    # print('sorted boxes', sorted_boxes)
    save_im_final_nms_featureMap = save_path + '/' + file_name[0] + '.png'
    cv2.imwrite(save_im_final_nms_featureMap, img_rgb)

boxes[0] [411 279 377 188]
boxes[1] [ 31 295 360 165]
